In [1]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim import SGD

from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((100,100)),
    transforms.RandomHorizontalFlip(),
    transforms.Grayscale(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5], [0.5])

])

In [4]:
#Dataloader

#Path for training and testing directory
train_path = 'C:/Users/Dell/Downloads/GSASLdatasets/train/'
test_path = 'C:/Users/Dell/Downloads/GSASLdatasets/test/'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=32, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [5]:

#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [6]:
print(classes)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [32]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=29):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (32,1,100,100)
        
        self.conv1=nn.Conv2d(in_channels=1,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (32,12,100,100)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (32,12,100,100)
        self.relu1=nn.ReLU()
        #Shape= (32,12,100,100)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (32,12,50,50)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (32,20,50,50)
        self.relu2=nn.ReLU()
        #Shape= (32,20,50,50)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (32,32,50,50)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (32,32,50,50)
        self.relu3=nn.ReLU()
        #Shape= (32,32,50,50)
        
        
        self.fc=nn.Linear(in_features=50 * 50 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*50*50)
            
            
        output=self.fc(output)
            
        return output
            

In [39]:
model=ConvNet(num_classes=29).to(device)

In [40]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [41]:
num_epochs=20

In [42]:
num_epochs=10
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [43]:
print(train_count,test_count)

69600 8700


In [44]:
# Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    

Epoch: 0 Train Loss: tensor(1.4307) Train Accuracy: 0.6716522988505748 Test Accuracy: 0.8529885057471265
Epoch: 1 Train Loss: tensor(0.2980) Train Accuracy: 0.9043247126436782 Test Accuracy: 0.9242528735632184
Epoch: 2 Train Loss: tensor(0.1822) Train Accuracy: 0.9417385057471265 Test Accuracy: 0.9447126436781609
Epoch: 3 Train Loss: tensor(0.1345) Train Accuracy: 0.9570114942528736 Test Accuracy: 0.9252873563218391
Epoch: 4 Train Loss: tensor(0.1128) Train Accuracy: 0.9645402298850575 Test Accuracy: 0.9402298850574713
Epoch: 5 Train Loss: tensor(0.0964) Train Accuracy: 0.9695545977011494 Test Accuracy: 0.9611494252873564
Epoch: 6 Train Loss: tensor(0.0879) Train Accuracy: 0.9727155172413793 Test Accuracy: 0.954367816091954
Epoch: 7 Train Loss: tensor(0.0811) Train Accuracy: 0.9751005747126437 Test Accuracy: 0.9573563218390805
Epoch: 8 Train Loss: tensor(0.0748) Train Accuracy: 0.9767528735632184 Test Accuracy: 0.9554022988505747
Epoch: 9 Train Loss: tensor(0.0671) Train Accuracy: 0.97